## Monitor transactions using anomaly detection model.     
---
**NOTE**: 

In real life scenarios financial transaction are dynamically evolving graphs. Performing anomaly detection inference on graph embeddings in live Transaction Monitoring Systems will require to update the graph and node representations after new transactions arrive. Recomputing entire graph for every newly arrived transaction will lead to unaxeptable delayes and even monitoring system failure. This problem  will be more sever if large amount of updates happen in a short time window.

Contact us at Logical Clocks and we will help you to setup end to end graph based deep anomaly detection for live Transaction Monitoring Systems. 

---

In [2]:
spark

In [3]:
import json
import numpy as np

![Image7-Monitor.png](./images/models.gif)

# Query Model Repository for best anomaly detection model

In [4]:
import hsml

conn = hsml.connection()
mr = conn.get_model_registry()

MODEL_NAME="ganAml"
EVALUATION_METRIC="loss"

Connected. Call `.close()` to terminate connection gracefully.

In [5]:
best_model = mr.get_best_model(MODEL_NAME, EVALUATION_METRIC, "min")

In [6]:
print('Model name: ' + best_model.name)
print('Model version: ' + str(best_model.version))
print(best_model.training_metrics)

Model name: ganAml
Model version: 1
{'loss': '-2.9802322387695312E-8'}

### Create the deployment
Here, we fetch the model we want from the model registry and define a configuration for the deployment. For the configuration, we need to specify the serving type (default or KFserving) and in this case, since we use default serving and an sklearn model, we need to give the location of the prediction script.

In [13]:
import hsml
from hsml.predictor_config import PredictorConfig

conn = hsml.connection()
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.

In [ ]:
# Create serving instance
SERVING_NAME = "ganAml"

# Use the model name from notebook 4, where we registered the model
model = mr.get_model(best_model.name, version=best_model.version)

# Give it any name you want
model.deploy(name=SERVING_NAME) #, model_server="PYTHON", serving_tool="DEFAULT"

In [12]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

ganAml

### Check Model Serving for active servings
![Image7-Monitor.png](./images/servings.gif)

In [21]:
# Get serving status
serving.get_status(MODEL_NAME)

'Running'

# Start Model Serving Server

In [22]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

In [23]:
import time
while serving.get_status(MODEL_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

# Retrieve serving vectors and send prediction requests to the served model using Hopsworks REST API

In [24]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [25]:
td_meta = fs.get_training_dataset("gan_non_sar_training_df", 1)
#`init_prepared_statement` method is needed to get serving_keys in case `get_serving_vector` has not beed called yet. This is not necessary for `get_serving_vector` method itself
td_meta.init_prepared_statement() 
td_meta.serving_keys

{'id'}

In [31]:
ids_to_score = ["0016359b", 
                "001dcc27", 
                "0054a022", 
                "00d6b609", 
                "00e14860", 
                "00e39a1b", 
                "014ed5cb", 
                "01ce3306", 
                "01fa19ae", 
                "01fa1d01", 
                "036dce03", 
                "03e09be4", 
                "04b23f4b"]

In [32]:
def model_server(model_name, input):
    data = {"signature_name": "serving_default", "inputs": [input]}
    return serving.make_inference_request(model_name, data)['outputs']

In [33]:
for node_id in ids_to_score:
    serving_vector = td_meta.get_serving_vector({'id': node_id})
    print(serving_vector)
    #model_server(MODEL_NAME, serving_vector[0])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]